Let's try to query all the archiveTypes present on the wiki. This information is available through two properties:

- ArchiveType present at the dataset level (not part of core)
- ProxyArchiveType available at the variable level (part of core)

This is the query from core:

In [125]:
import json
import requests

url = "http://wiki.linked.earth/store/db/query"

query = """PREFIX core: <http://linked.earth/ontology#>
PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT distinct ?a 
WHERE {
{
    ?w core:proxyArchiveType ?t.
    ?t rdfs:label ?a
}
}"""

response = requests.post(url, data = {'query': query})
res = json.loads(response.text)
print(response)
print("The following archive types are available on the wiki:")    
for item in res['results']['bindings']:
    print ("*" + item['a']['value'])

<Response [200]>
The following archive types are available on the wiki:
*Coral
*GlacierIce
*LakeSediment
*MarineSediment
*Speleothem


Doesn't return any of the ProxyArchiveTypes.

This is the response from wiki: 

In [126]:
import json
import requests

url = "http://wiki.linked.earth/store/db/query"

query = """PREFIX core: <http://linked.earth/ontology#>
PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT distinct ?a 
WHERE {
{
    ?dataset wiki:Property-3AArchiveType ?a.
}
}"""

response = requests.post(url, data = {'query': query})
res = json.loads(response.text)
print(response)
print("The following archive types are available on the wiki:")    
for item in res['results']['bindings']:
    print ("*" + item['a']['value'])

<Response [200]>
The following archive types are available on the wiki:
*tree
*marine sediment
*speleothem
*lake sediment
*glacier ice
*borehole
*documents
*hybrid
*coral
*bivalve
*sclerosponge


Trying to go around the core issue is not easy because I don't know how to fix the URL interpretation of ':', '_', '(',')'

Trying to query through the wiki prefix but not sure how to deal with special characters in URL. See below:

In [127]:
# DataSetName = ["Aus-StewartIsland.D'Arrigo.1996", "Arc-BigRoundLake.Thomas.2009"]
DataSetName = ["ME0005A-43JC.Benway.2006"]

In [128]:
import json
import requests
import pandas as pd

url = "http://wiki.linked.earth/store/db/query"

query = """PREFIX core: <http://linked.earth/ontology#>
PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?dataset ?a
WHERE {"""

query+= "VALUES ?b {"
for item in DataSetName:
    query +="\""+item+"\" "
query+= "}\n"

query+= "?dataset wiki:Property-3AName%5F%28L%29 ?b. \n"
query+= ' ?dataset wiki:Property-3AArchiveType ?a \n}'

response = requests.post(url, data = {'query': query})
res = json.loads(response.text)

print(response,res)
#archiveType = []
#for item in res['results']['bindings']:
#    archiveType.append(item['a']['value'])

#results = pd.DataFrame({'dataset name': DataSetName,'archive type': archiveType})
#results

<Response [200]> {'head': {'vars': ['dataset', 'a']}, 'results': {'bindings': []}}


#### Get all the LiPD filenames from the wiki

In [129]:
import json
import requests

url = "http://wiki.linked.earth/store/db/query"

query = """PREFIX core: <http://linked.earth/ontology#>
PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
SELECT ?ds ?name
WHERE {
  ?ds a core:Dataset .
  ?ds core:name ?name
}"""

response = requests.post(url, data = {'query': query})
res = json.loads(response.text)

files_in_wiki = set()
for item in res['results']['bindings']:
    files_in_wiki.add(item['name']['value'])

In [130]:
files_in_wiki

{'A7.Oppo.2005',
 'Afr-ColdAirCave.Sundqvist.2013',
 'Afr-LakeMalawi.Powers.2011',
 'Afr-LakeTanganyika.Tierney.2010',
 'Afr-P178-15P.Tierney.2015',
 'Ant-BerknerIsland.Mulvaney.2002',
 'Ant-CoastalDML.Thamban.2012',
 'Ant-DSS.DahlJensen.1999',
 'Ant-DSS.Moy.2012',
 'Ant-DomeC.Jouzel.2001',
 'Ant-DomeC.Stenni.2001',
 'Ant-DomeF1993.Uemura.2014',
 'Ant-DomeF2001.Uemura.2008',
 'Ant-Ferrigno.Thomas.2013',
 'Ant-JamesRossIsland.Mulvaney.2013',
 'Ant-MES.Rhodes.2012',
 'Ant-PlateauRemote.Mosley-Thompson.2013',
 'Ant-SipleStation.Mosley-Thompson.1990',
 'Ant-SiteDML05.Graf.2002',
 'Ant-SiteDML07.Graf.2002',
 'Ant-SiteDML17.Graf.2002',
 'Ant-TALDICE.Stenni.2010',
 'Ant-TalosDome.Stenni.2002',
 'Ant-TaylorDome.Steig.2000',
 'Ant-US-ITASE-2000-1.Steig.2013',
 'Ant-US-ITASE-2002-4.Steig.2013',
 'Ant-VLG.Bertler.2011',
 'Ant-Vostok.Ekaykin.2014',
 'Ant-WAIS-Divide.Severinghaus.2012',
 'Ant-WDC05A.Steig.2013',
 'Ant-WDC06A.Steig.2013',
 'Arc-Agassiz.Vinther.2008',
 'Arc-AkademiiNaukIceCap.Opel.20

#### For a filename , get the archive Type

In [131]:
DataSetName = ["Aus-StewartIsland.D'Arrigo.1996", "Arc-BigRoundLake.Thomas.2009"]
# DataSetName = ["Afr-LakeMalawi.Powers.2011"]

These datasets have an archiveType.  
http://wiki.linked.earth/Aus-StewartIsland.D%27Arrigo.1996  
http://wiki.linked.earth/Arc-BigRoundLake.Thomas.2009

In [132]:
import json
import requests
import pandas as pd

url = "http://wiki.linked.earth/store/db/query"

query = """PREFIX core: <http://linked.earth/ontology#>
PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?dataset ?a ?pot
WHERE {"""

query+= "VALUES ?b {"
for item in DataSetName:
    query +="\""+item+"\" "
query+= "}\n"

query+= "?dataset core:name ?b. \n"
query+= ' ?dataset wiki:Property-3AArchiveType ?a. \n}'
# query+= ' ?dataset core:includesPaleoData ?pd. \n}'
# query+= ' ?pd core:foundInMeasurementTable ?table. \n}'
# query+= ' ?table core:includesVariable ?var . \n}'
# query+= ' ?var core:proxyObservationType ?pot \n}'

response = requests.post(url, data = {'query': query})
res = json.loads(response.text)

#archiveType = []
#for item in res['results']['bindings']:
#    archiveType.append(item['a']['value'])

#results = pd.DataFrame({'dataset name': DataSetName,'archive type': archiveType})
#results

print(response, res)

<Response [200]> {'head': {'vars': ['dataset', 'a', 'pot']}, 'results': {'bindings': [{'dataset': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/Aus-2DStewartIsland.D-27Arrigo.1996'}, 'a': {'type': 'literal', 'value': 'tree'}}, {'dataset': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/Arc-2DBigRoundLake.Thomas.2009'}, 'a': {'type': 'literal', 'value': 'lake sediment'}}]}}


To see how many lipd files have paleodata from all the files in the wiki

In [133]:
import os
import sys
import json
import requests
import urllib

# datasetname = "Afr-LakeMalawi.Powers.2011"
datasetname = "Arc-BigRoundLake.Thomas.2009"

endpoint = "http://wiki.linked.earth/store/db/query" # Query Metadata
wikiapi = "http://wiki.linked.earth/wiki/api.php" # Fetch file data

query = """PREFIX core: <http://linked.earth/ontology#>
PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
SELECT ?name ?data ?pot
WHERE {
  ?ds a core:Dataset .
  ?ds core:name ?name .
  ?ds core:includesPaleoData ?data .
  ?data core:foundInMeasurementTable ?table .
  ?table core:includesVariable ?var .
}"""
# ?data core:foundInMeasurementTable ?table .
#   ?table core:includesVariable ?var .
#   ?var core:proxyObservationType ?pot.

# ?table core:hasFileName ?filename

# ?ds core:name '""" + datasetname + """' .
response = requests.post(endpoint, data = {'query': query})
res = json.loads(response.text)
# print(response, res)

files_with_paleo = set()
for item in res['results']['bindings']:
    files_with_paleo.add(item['name']['value'])

In [134]:
len(files_with_paleo)

157

In [144]:
file_list = list(files_with_paleo.intersection(files_in_wiki))
print(file_list)
len(file_list)

['NAm-SnowBowlSanFranciscoPeak.Briffa.2002', 'Arc-PolarUrals.Wilson.2015', 'Aus-MtRead.Cook.2006', 'Asi-KYRG008.Shiyatov.2013', 'Asi-BT010.Cook.2010', "NAm-SewardComposite.D'Arrigo.2006", 'Ocn-AbrahamReef.Druffel.1999', 'Ocn-BermudaSouthShore.Goodkin.2008', 'Ocn-CH07-98-MC-22.Saenger.2011', 'Asi-KYRG004.Esper.2007', 'Ocn-JacafFjord.Sepúlveda.2009', 'Asi-PAKI035.Cook.2013', 'NAm-Bonif.Schweingruber.1996', 'NAm-SunwaptaPass.Schweingruber.1996', 'NAm-SpruceCreek.Church.1981', 'SAm-CentralAndes11.PAGES2k.2011', 'Arc-StoreggaSlide.Sejrup.2011', "NAm-SouthernAlaskacomposite.D'Arrigo.2006", 'Ocn-SWCoastOfIndia.Saraswat.2013', 'Asi-CHIN017.Li.2010', 'M77-2_056.Nurnberg.2015', 'Asi-CHIN044.Brauning.2013', 'Eur-NorthernSpain.Martín-Chivelet.2011', 'Asi-BT005.Cook.2010', 'Ocn-AqabaJordanAQ19.Heiss.1999', 'Ocn-CariacoBasin.Black.2007', 'MD95-2043.Cacho.1999', 'Eur-SpanishPyrenees.Dorado-Linan.2012', 'NAm-CraterLakeNE.Briffa.2002', 'Asi-NEPA015.Krusic.2013', "Aus-StewartIsland.D'Arrigo.1996", 'Ocn-

157

In [145]:
import pandas as pd
master_table = pd.DataFrame(columns = ['dataset name', 'archiveType', 'proxyObservationType', 'units', 'proxySystem'])

Get all the proxyObsTypes for files that have paleodata

In [147]:
import os
import sys
import json
import requests
import urllib

no_content_files = []
no_paleodata = []
# datasetname = "Afr-LakeMalawi.Powers.2011"
# datasetname = "Afr-LakeTanganyika.Tierney.2010"
# file_list=['NAm-SnowBowlSanFranciscoPeak.Briffa.2002', "NAm-SewardComposite.D'Arrigo.2006", 'Ocn-AbrahamReef.Druffel.1999']
# file_list = ['Arc-BigRoundLake.Thomas.2009']

endpoint = "http://wiki.linked.earth/store/db/query" # Query Metadata
wikiapi = "http://wiki.linked.earth/wiki/api.php" # Fetch file data

for datasetname in file_list:
    query = """PREFIX core: <http://linked.earth/ontology#>
    PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
    SELECT ?name ?a ?potName ?uni ?psys
    WHERE {
    
       # binding defaults
      bind("NA" as ?default_units)
      bind("NA" as ?default_psys)
      
      ?ds a core:Dataset .
      ?ds core:name ?name .
      ?ds core:name '""" + datasetname + """' .
      ?ds wiki:Property-3AArchiveType ?a.
      ?ds core:includesPaleoData ?data .
      ?data core:foundInMeasurementTable ?table .
      ?table core:includesVariable ?var .
      ?var core:proxyObservationType ?pot .
      ?var core:name ?potName .
      
      OPTIONAL {
          ?var core:hasUnits ?wiki_uni .
          ?var core:hasProxySystem ?wiki_psys .
      }
      
      # either take ?wiki_uni/wiki_psys value or default and bind it as the result ?uni/psys
      bind(coalesce(?wiki_uni, ?default_units) as ?uni)
      bind(coalesce(?wiki_psys, ?default_psys) as ?psys)
      
    }"""

    try:
        response = requests.post(endpoint, data = {'query': query})
        res = json.loads(response.text)
        print(response, res)
        print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
        
        if not res['results']['bindings']:
            no_content_files.append(datasetname)
        for item in res['results']['bindings']:
            df = pd.DataFrame({'dataset name':item['name']['value'], 'archiveType': item['a']['value'], 'proxyObservationType':item['potName']['value'], 'units':item['uni']['value'], 'proxySystem':item['psys']['value']}, index=[0])
            master_table = master_table.append(df, ignore_index = True)
    except:
        no_paleodata.append(datasetname)
        print('Dataset does not have Paleodata :(')


master_table.to_csv(r'D:\\annotating_paleoclimate_data\\proxyObsTypeWiki.csv', sep = ',', encoding = 'utf-8',index = False)

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'NAm-SnowBowlSanFranciscoPeak.Briffa.2002'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'segment'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-SnowBowlSanFranciscoPeak.Briffa.2002'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'MXD'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.MXDDefaultSensor.MXD'}}, {'name': {'type': 'literal', 'value': 'NAm-SnowBowlSanFranciscoPeak.Briffa.2002'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'residual'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'ps

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'NAm-SpruceCreek.Church.1981'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'EPS'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-SpruceCreek.Church.1981'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'ARS'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-SpruceCreek.Church.1981'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'SD'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/

Dataset does not have Paleodata :(
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': []}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': []}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': []}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': []}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'Ant-US-ITASE-200

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'Asi-MONG030.Jacoby.2010'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'trsgi'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.TRWDefaultSensor.TRW'}}, {'name': {'type': 'literal', 'value': 'Asi-MONG030.Jacoby.2010'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'trsgi'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.TRWDefaultSensor.TRW'}}]}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'Asi-Hushre.PA

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'NAm-RedMountainPassSilverton.Graybill.1994'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'sampleCount'}, 'uni': {'type': 'literal', 'value': 'count'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-RedMountainPassSilverton.Graybill.1994'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'segment'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-RedMountainPassSilverton.Graybill.1994'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'ringWidth'}, 'uni': {'type': 'literal', 'value':

...

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'NAm-Sugarloaf.Kenigsberg.2013'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'trsgi'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.TRWDefaultSensor.TRW'}}, {'name': {'type': 'literal', 'value': 'NAm-Sugarloaf.Kenigsberg.2013'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'trsgi'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.TRWDefaultSensor.TRW'}}]}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': []}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': []}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'Afr-LakeMalawi.Powers.2011'}, 'a': {'type': 'literal', 'value': 'lake sediment'}, 'potName': {'type': 'literal', 'value': 'core'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'literal', 'value': 'NA'}}, {'name': {'type': 'literal', 'value': 'Afr-LakeMalawi.Powers.2011'}, 'a': {'type': 'literal', 'value': 'lake sediment'}, 'potName': {'type': 'literal', 'value': 'TEX86'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.LakeSediment.TEX86DefaultSensor.TEX86'}}]}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'NAm-BlanchardRiver.Luckman.2013'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'sampleCount'}, 'uni': {'type': 'literal', 'value': 'count'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-BlanchardRiver.Luckman.2013'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'ringWidth'}, 'uni': {'type': 'literal', 'value': 'cm'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-BlanchardRiver.Luckman.2013'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'residual'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'NAm-wa071.Peterson.1994'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'ringWidth'}, 'uni': {'type': 'literal', 'value': 'cm'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-wa071.Peterson.1994'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'corrs'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.DefaultSensor'}}, {'name': {'type': 'literal', 'value': 'NAm-wa071.Peterson.1994'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'RBar'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Sp

<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'Asi-RUSS236.Sano.2009'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'trsgi'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.TRWDefaultSensor.TRW'}}, {'name': {'type': 'literal', 'value': 'Asi-RUSS236.Sano.2009'}, 'a': {'type': 'literal', 'value': 'tree'}, 'potName': {'type': 'literal', 'value': 'trsgi'}, 'uni': {'type': 'literal', 'value': 'NA'}, 'psys': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/ProxySystem.Wood.TRWDefaultSensor.TRW'}}]}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
<Response [200]> {'head': {'vars': ['name', 'a', 'potName', 'uni', 'psys']}, 'results': {'bindings': []}}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

In [110]:
print(len(no_content_files))
no_content_files

75


['Arc-PolarUrals.Wilson.2015',
 'Aus-MtRead.Cook.2006',
 'Asi-KYRG008.Shiyatov.2013',
 'Asi-BT010.Cook.2010',
 'Ocn-BermudaSouthShore.Goodkin.2008',
 'Asi-KYRG004.Esper.2007',
 'Ocn-JacafFjord.Sepúlveda.2009',
 'Asi-PAKI035.Cook.2013',
 'Arc-StoreggaSlide.Sejrup.2011',
 'Asi-CHIN017.Li.2010',
 'Eur-NorthernSpain.Martín-Chivelet.2011',
 'Asi-BT005.Cook.2010',
 'Ocn-AqabaJordanAQ19.Heiss.1999',
 'MD95-2043.Cacho.1999',
 'Asi-NEPA015.Krusic.2013',
 'Ocn-Rarotonga_d18O99.Linsley.2006',
 'Ocn-Mafia.Damassa.2006',
 'Asi-NEPA030.Krusic.2013',
 'Ocn-CariacoBasin.Lea.2003',
 'Asi-ZhejiangAndFujian.Zhang.1980',
 'Ocn-PedradeLume-CapeVerdeIslands.Moses.2006',
 'Asi-CHIN037.Fan.2013',
 'Ocn-ODP984.Came.2007',
 'Ocn-IfatyTul3.Zinke.2014',
 'Asi-MONG033.Jacoby.2010',
 'Asi-HYGJUP.PAGES2k.2013',
 'Asi-GuangdongAndGuangxi.Zhang.1980',
 'Arc-MD992275.Jiang.2005',
 'Arc-LakeDonardBaffinIsland.Moore.2001',
 'Asi-CHIN018.Li.2010',
 'DSDP502.deMenocal.1992',
 'Arc-SoperLake.Hughen.2000',
 'Ant-DomeC.Stenni

In [143]:
import os
import sys
import json
import requests
import urllib

# datasetname = "Afr-LakeMalawi.Powers.2011"
datasetname = "Arc-BigRoundLake.Thomas.2009"

endpoint = "http://wiki.linked.earth/store/db/query" # Query Metadata
wikiapi = "http://wiki.linked.earth/wiki/api.php" # Fetch file data

query = """PREFIX core: <http://linked.earth/ontology#>
PREFIX wiki: <http://wiki.linked.earth/Special:URIResolver/>
SELECT ?name ?data ?table ?pot
WHERE {
  ?ds a core:Dataset .
  ?ds core:name ?name .
  ?ds core:name '""" + datasetname + """' .
  ?ds core:includesPaleoData ?data .
  ?data core:foundInMeasurementTable ?table .
  
}"""
# ?data core:foundInMeasurementTable ?table .
#   ?table core:includesVariable ?var .
#   ?var core:proxyObservationType ?pot.

response = requests.post(endpoint, data = {'query': query})
res = json.loads(response.text)
print(response, res)

<Response [200]> {'head': {'vars': ['name', 'data', 'table', 'pot']}, 'results': {'bindings': [{'name': {'type': 'literal', 'value': 'Arc-BigRoundLake.Thomas.2009'}, 'data': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/Arc-2DBigRoundLake.Thomas.2009.PaleoData1'}, 'table': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/Arc-2DBigRoundLake.Thomas.2009.paleo1measurement1'}}, {'name': {'type': 'literal', 'value': 'Arc-BigRoundLake.Thomas.2009'}, 'data': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/Arc-2DBigRoundLake.Thomas.2009.PaleoData2'}, 'table': {'type': 'uri', 'value': 'http://wiki.linked.earth/Special:URIResolver/Arc-2DBigRoundLake.Thomas.2009.paleo2measurement1'}}]}}
